## **CLASIFICATION**

In [4]:
install.packages('VGAM')

Updating HTML index of packages in '.Library'
Making 'packages.html' ... done


In [5]:
library(tidyverse)
library(VGAM)
library(caret)

Loading required package: stats4
Loading required package: splines

Attaching package: ‘VGAM’

The following object is masked from ‘package:tidyr’:

    fill

Loading required package: lattice

Attaching package: ‘caret’

The following object is masked from ‘package:VGAM’:

    predictors

The following object is masked from ‘package:purrr’:

    lift



In [6]:
# Loading data
Airlines <- read.csv("data/AirlineDelay.csv")

In [7]:
Airlines$DelayClass = factor(ifelse(Airlines$TotalDelay == 0, "No Delay", ifelse
                                    (Airlines$TotalDelay >= 30, "Major Delay", "Minor Delay")))
levels(Airlines$DelayClass)

[1] "Major Delay" "Minor Delay" "No Delay"

In [8]:
# Very important: remove TotalDelay variable!
# Why?
Airlines$TotalDelay = NULL

# split between training and testing sets
spl = createDataPartition(Airlines$DelayClass, p = 0.8, list = FALSE)  # 80% for training

AirlinesTrain = Airlines[spl,]
AirlinesTest = Airlines[-spl,]

# How many flights in the trainining dataset Airlines had no delay, minor delay, major delay respectively?
table(AirlinesTrain$DelayClass)

# By route:
table(AirlinesTrain$DelayClass, AirlinesTrain$Flight)

# Logistic regression:

log.fit = vglm(DelayClass ~ HistoricallyLate, family=multinomial(refLevel=1), data=AirlinesTrain)
summary(log.fit)

# interpretation:
#  for every one unit change in HistoricallyLate, the log odds of MinorDelay (vs MajorDelay) decreases by -1.5
#  for every one unit change in HistoricallyLate, the log odds of NoDelay (vs MajorDelay) decreases by -3

confint(log.fit)

# Odd ratios
exp(coef(log.fit))
# the expected increase in the odds of MinorDelay (vs MajorDelay) for each unit change in HistoricallyLate. 
# An odds ratio of 1 indicates no change, whereas an odds ratio of 2 indicates a doubling, and 0.5 indicates halving, etc.

# predicting the testing set
prob.test = predict(log.fit, newdata=AirlinesTest, type="response")
prob.test
# output are probabilities, no labels



Major Delay Minor Delay    No Delay 
       1278        2477        3751 

             
              ATL-LAX ATL-ORD LAX-ATL LAX-ORD ORD-ATL ORD-LAX
  Major Delay     144     174     101     294     189     376
  Minor Delay     426     227     301     651     229     643
  No Delay        601     468     758     830     455     639


Call:
vglm(formula = DelayClass ~ HistoricallyLate, family = multinomial(refLevel = 1), 
    data = AirlinesTrain)


Pearson residuals:
                      Min      1Q  Median    3Q   Max
log(mu[,2]/mu[,1]) -1.863 -0.3916 -0.3503 1.252 1.417
log(mu[,3]/mu[,1]) -2.211 -0.6983 -0.1674 0.843 2.083

Coefficients: 
                   Estimate Std. Error z value Pr(>|z|)    
(Intercept):1       1.02751    0.05167  19.885   <2e-16 ***
(Intercept):2       1.71738    0.04890  35.117   <2e-16 ***
HistoricallyLate:1 -1.51486    0.15223  -9.951   <2e-16 ***
HistoricallyLate:2 -3.10025    0.15550 -19.937   <2e-16 ***
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Number of linear predictors:  2 

Names of linear predictors: log(mu[,2]/mu[,1]), log(mu[,3]/mu[,1])

Residual deviance: 14791.28 on 15008 degrees of freedom

Log-likelihood: -7395.642 on 15008 degrees of freedom

Number of iterations: 4 

No Hauck-Donner effect found in any of the estimates

Reference group is leve

,2.5 %,97.5 %
(Intercept):1,0.9262292,1.128783
(Intercept):2,1.6215254,1.813226
HistoricallyLate:1,-1.8132118,-1.216500
HistoricallyLate:2,-3.4050312,-2.795473


(Intercept):1      (Intercept):2 HistoricallyLate:1 HistoricallyLate:2 
        2.79408878         5.56989087         0.21983982         0.04503786

,Major Delay,Minor Delay,No Delay
9,0.2142229,0.3612503,0.4245268
14,0.2541405,0.3709879,0.3748716
24,0.2489511,0.3700257,0.3810232
30,0.2277680,0.3651786,0.4070533
33,0.2265171,0.3648440,0.4086389
39,0.2142229,0.3612503,0.4245268
41,0.2077901,0.3591408,0.4330691
48,0.1979741,0.3556009,0.4464250
50,0.1935641,0.3538790,0.4525569
58,0.1920149,0.3532540,0.4547311


In [9]:

# How to predict the labels for delay?
# We can apply the rule of maximum probability
pred.test <- as.factor(levels(Airlines$DelayClass)[max.col(prob.test)])
pred.test

# summarize accuracy (confusion matrix) for a given probability rule (maximum in this case)
# predictions in rows, true values in columns (but we can change the order)
ConfMat = table(AirlinesTest$DelayClass,pred.test)
ConfMat

n = length(AirlinesTest$DelayClass)
prop.errors <- (n - sum(diag(ConfMat))) / n
prop.errors

accuracy <- sum(diag(ConfMat)) / n
accuracy

# Better with Caret...
confusionMatrix(AirlinesTest$DelayClass,pred.test)

# Is it a reasonable accuracy?
# We are only using one predictor...

# Perform some simple regressions to understand what is behind
for(j in c(1:22)){
  print(colnames(AirlinesTrain)[j])
  log.fit=vglm(AirlinesTrain$DelayClass ~ AirlinesTrain[,j], family=multinomial(refLevel=3))
  print(summary(log.fit))
  readline(prompt="Press [enter] to continue ")
}

log.fit = vglm(DelayClass ~ ., family=multinomial(refLevel=1), data=AirlinesTrain)
summary(log.fit)

# Each model can be automatically tuned and evaluated 
# In this case, we are goint to use 4 repeats of 5-fold cross validation
ctrl <- trainControl(method = "repeatedcv", 
                     repeats = 4,
                     number = 5)
# we can also choose bootstrap, LOOCV, etc.
# ctrl = trainControl(method = 'LGOCV', p = 0.8, number = 30)
#  means 30 repeated training/test splits

# We have many predictors, hence use penalized logistic regression
lrFit <- train(DelayClass ~ ., 
                method = "glmnet",
                family = "multinomial",
                data = AirlinesTrain,
                preProcess = c("center", "scale"),
                trControl = ctrl)
print(lrFit)
lrPred = predict(lrFit, AirlinesTest)
confusionMatrix(AirlinesTest$DelayClass,lrPred)
lr_imp <- varImp(lrFit, scale = F)
plot(lr_imp, scales = list(y = list(cex = .95)))

ldaFit <- train(DelayClass ~ ., 
                #method = "lda", 
                method = "PenalizedLDA", 
                #method = "sparseLDA", 
                #method = "stepLDA", 
                data = AirlinesTrain,
                preProcess = c("center", "scale"),
                trControl = ctrl)
print(ldaFit)
ldaPred = predict(ldaFit, AirlinesTest)
confusionMatrix(AirlinesTest$DelayClass,ldaPred)
lda_imp <- varImp(ldaFit, scale = F)
plot(lda_imp, scales = list(y = list(cex = .95)))

nbFit <- train(DelayClass ~ ., method = "nb", 
                data = AirlinesTrain,
                preProcess = c("center", "scale"),
                trControl = ctrl)
print(nbFit)
nbPred = predict(nbFit, AirlinesTest)
confusionMatrix(AirlinesTest$DelayClass,nbPred)
nb_imp <- varImp(nbFit, scale = F)
plot(nb_imp, scales = list(y = list(cex = .95)))

rfFit <- train(DelayClass ~ ., method = "rf", 
               data = AirlinesTrain,
               preProcess = c("center", "scale"),
               trControl = ctrl)
print(rfFit)
rfPred = predict(rfFit, AirlinesTest)
confusionMatrix(AirlinesTest$DelayClass,rfPred)
rf_imp <- varImp(rfFit, scale = F)
plot(rf_imp, scales = list(y = list(cex = .95)))

# Is 50% or 60% of accuracy good enough?

# What is the accuracy on a benchmark model that predicts the most 
# frequent outcome (No Delay) for all observations?
table(AirlinesTrain$DelayClass)
obs <- max(table(AirlinesTest$DelayClass))
# Accuracy:
obs/nrow(AirlinesTest)

# Note the benchmark is not so bad... Too much noise...

# We can reduce the noise, or increase the accuracy by considering just two classes: delay or no delay
# But the information (or precision) of the output will be weaker or less practical...

Airlines$DelayClass = factor(ifelse(Airlines$DelayClass == "No Delay", "No Delay", "Delay"))
levels(Airlines$DelayClass)

AirlinesTrain = Airlines[spl,]
AirlinesTest = Airlines[-spl,]

table(AirlinesTrain$DelayClass)

# We have many predictors, hence use penalized logistic regression
lrFit <- train(DelayClass ~ ., 
               method = "glmnet",
               family = "multinomial",
               data = AirlinesTrain,
               preProcess = c("center", "scale"),
               trControl = ctrl)
print(lrFit)
lrPred = predict(lrFit, AirlinesTest)
confusionMatrix(AirlinesTest$DelayClass,lrPred)
lr_imp <- varImp(lrFit, scale = F)
plot(lr_imp, scales = list(y = list(cex = .95)))

# Now the accuracy is around 70%
# The other 30% are errors

# But are all the errors equally important?
# Usually not...

lrProb = predict(lrFit, newdata=AirlinesTest, type="prob")
lrProb = lrProb[1:nrow(AirlinesTest),]
threshold = 0.5
lrPred = rep("Delay", nrow(AirlinesTest))
lrPred[which(lrProb[,2] < threshold)] = "No Delay"
confusionMatrix(AirlinesTest$DelayClass,lrPred)

# Changing the threshold allows us to control better one of type the errors (by increasing the other one)
threshold = 0.4
lrPred = rep("Delay", nrow(AirlinesTest))
lrPred[which(lrProb[,2] < threshold)] = "No Delay"
confusionMatrix(AirlinesTest$DelayClass,lrPred)




[1] No Delay    No Delay    No Delay    No Delay    No Delay    No Delay   
   [7] No Delay    No Delay    No Delay    No Delay    No Delay    No Delay   
  [13] No Delay    No Delay    No Delay    No Delay    Minor Delay No Delay   
  [19] Minor Delay No Delay    No Delay    No Delay    No Delay    No Delay   
  [25] No Delay    No Delay    No Delay    No Delay    No Delay    No Delay   
  [31] No Delay    No Delay    Minor Delay Minor Delay Minor Delay No Delay   
  [37] No Delay    No Delay    No Delay    No Delay    No Delay    No Delay   
  [43] No Delay    No Delay    No Delay    No Delay    No Delay    No Delay   
  [49] No Delay    No Delay    No Delay    No Delay    No Delay    No Delay   
  [55] No Delay    No Delay    Major Delay Minor Delay No Delay    No Delay   
  [61] No Delay    No Delay    No Delay    Minor Delay No Delay    No Delay   
  [67] No Delay    No Delay    No Delay    No Delay    No Delay    No Delay   
  [73] No Delay    No Delay    Minor Delay No Delay    No Delay    Minor Delay
  [79] Minor Delay Minor Delay Minor Delay Minor Delay Minor Delay Minor Delay
  [85] Minor Delay Minor Delay Minor Delay Minor Delay Minor Delay Minor Delay
  [91] Minor Delay No Delay    No Delay    Minor Delay No Delay    Minor Delay
  [97] No Delay    No Delay    No Delay    No Delay    No Delay    No Delay   
 [103] No Delay    No Delay    No Delay    No Delay    No Delay    Minor Delay
 [109] Minor Delay No Delay    Minor Delay No Delay    No Delay    No Delay   
 [115] No Delay    No Delay    No Delay    No Delay    No Delay    No Delay   
 [121] No Delay    Minor Delay Minor Delay Minor Delay No Delay    Minor Delay
 [127] Minor Delay Minor Delay Minor Delay Minor Delay Minor Delay Minor Delay
 [133] Minor Delay Minor Delay Minor Delay Minor Delay Minor Delay Minor Delay
 [139] Minor Delay No Delay    No Delay    No Delay    No Delay    No Delay   
 [145] No Delay    No Delay    No Delay    No Delay    Minor Delay No Delay   
 [151] No Delay    Minor Delay Minor Delay No Delay    No Delay    No Delay   
 [157] No Delay    No Delay    No Delay    No Delay    No Delay    No Delay   
 [163] No Delay    No Delay    No Delay    No Delay    No Delay    No Delay   
 [169] No Delay    No Delay    No Delay    No Delay    No Delay    No Delay   
 [175] No Delay    No Delay    No Delay    No Delay    No Delay    No Delay   
 [181] No Delay    No Delay    No Delay    No Delay    Minor Delay No Delay   
 [187] No Delay    No Delay    Minor Delay Minor Delay No Delay    No Delay   
 [193] Minor Delay Minor Delay Major Delay Minor Delay Minor Delay Minor Delay
 [199] Minor Delay Minor Delay Minor Delay Minor Delay Minor Delay Minor Delay
 [205] Minor Delay Minor Delay No Delay    No Delay    No Delay    No Delay   
 [211] No Delay    No Delay    No Delay    No Delay    No Delay    No Delay   
 [217] No Delay    No Delay    No Delay    No Delay    No Delay    No Delay   
 [223] No Delay    No Delay    No Delay    No Delay    No Delay    No Delay   
 [229] No Delay    No Delay    No Delay    No Delay    No Delay    No Delay   
 [235] No Delay    No Delay    No Delay    No Delay    No Delay    No Delay   
 [241] No Delay    No Delay    No Delay    No Delay    No Delay    No Delay   
 [247] No Delay    No Delay    Minor Delay Minor Delay No Delay    No Delay   
 [253] No Delay    No Delay    Major Delay No Delay    No Delay    No Delay   
 [259] No Delay    No Delay    No Delay    No Delay    No Delay    No Delay   
 [265] No Delay    No Delay    No Delay    No Delay    No Delay    Minor Delay
 [271] Minor Delay Minor Delay No Delay    No Delay    No Delay    No Delay   
 [277] No Delay    No Delay    No Delay    No Delay    No Delay    No Delay   
 [283] No Delay    No Delay    No Delay    No Delay    No Delay    No Delay   
 [289] No Delay    No Delay    No Delay    No Delay    No Delay    No Delay   
 [295] Minor Delay No Delay    No Delay    No Delay    No Delay    No Delay   
 [301] No Delay    No Delay    No Delay    No Delay  

             pred.test
              Major Delay Minor Delay No Delay
  Major Delay          26          84      209
  Minor Delay          33          81      505
  No Delay              2          50      885

[1] 0.4709333

[1] 0.5290667

ERROR: Error: package e1071 is required
